In [1]:
from Volatility import *

from dotenv import load_dotenv
load_dotenv()

import os

import math
import numpy as np
import datetime
from datetime import date

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px


# Import Libraries
import pandas as pd
import urllib
import requests
import plotly.graph_objects as go
pd.options.plotting.backend = "plotly"

##GVOL API ENDPOINTS 
from gvol import GVol
GVOL_API_KEY = os.getenv('GVOL_API_KEY_LITE') #VARIABLE CONTAINED IN .ENV FILE
#GVOL_API_KEY = 'GVOL_API_KEY_LITE'
gvol_client = GVol(header='gvol-lite',gvol_api_key=GVOL_API_KEY)

# Chart libraries + settings
#%matplotlib inline
#import matplotlib.pyplot as plt
# plt.style.use('seaborn-whitegrid')
pd.options.mode.chained_assignment = None  # default='warn' - disable some pandas warnings

In [7]:
now = datetime.datetime.utcnow().date()
now = str(now)

symbol = 'BTC'

##RETRIEVE PRICES
prices = gvol_client.SpotPricesLite(symbol=symbol)
prices = pd.json_normalize(prices['SpotPricesLite'])
prices['date'] = pd.to_datetime(prices['date'],unit='ms')
prices.set_index('date',inplace=True)
prices = prices
prices.columns = ['Curreny','Open','High','Low','Close']
prices.sort_index(inplace=True)


##RETRIEVE CONSTANT ATM IV
now = datetime.datetime.utcnow().date() + datetime.timedelta(days=1)
now = str(now)
    
iv = gvol_client.FixedMaturityAtm(exchange='deribit',symbol=symbol)
iv = pd.json_normalize(iv['FixedMaturityAtm'])
iv['date']= pd.to_datetime(iv['date'],unit='ms')
iv.set_index('date',inplace=True)
iv = iv.resample('1D').mean()


##CALCULATING RV CHOOSE YOUR ESTIMATOR HERE
window = (7,30,60,90,180)
###[raw,parkinson,garmanklass,yangzhang]
vol_estimator = parkinson

df = pd.DataFrame()
for days in window:
    df_days = vol_estimator(prices,window=days,clean=False)*100
    df = pd.concat([df,df_days],axis=1)

rv = df
col_names = ["rv" + str(day) for day in window]
rv.columns = col_names

rv_iv = rv.join(iv)
rv_iv = rv_iv.iloc[-30:]

In [9]:
##NOT VARIABLES DO NOT CHANGE
window = (['rv7','atm7'],['rv30','atm30'],['rv90','atm90'],['rv180','atm180'])

## COMMENT/DECOMMENT IF YOU WANNA SHIFT IV FORWARD
#shift = (7,30,90,180)
shift = (0,0,0,0)
###

s = 0
vol_max = 0
vol_min = 0
for time in (window):
    
    #DEFINE SUBPLOTS CHART
    fig = make_subplots(rows=2,cols=1,row_heights=[0.8, 0.2])
    
    rv_iv_time = rv_iv[time] 
    
    fig.add_trace(go.Scatter(x=rv_iv_time.index,y=rv_iv_time.iloc[:,0],name=rv_iv_time.iloc[:,0].name,fill='tozeroy'),row=1,col=1)
    fig.add_trace(go.Scatter(x=rv_iv_time.index,y=rv_iv_time.iloc[:,1].shift(shift[s]),name=rv_iv_time.iloc[:,1].name,fill='tozeroy'),row=1,col=1)

    premium = rv_iv_time.iloc[:,1].shift(shift[s]) - rv_iv_time.iloc[:,0]
    
    fig.add_trace(go.Bar(x=premium.index,y=premium,name='var premium'),row=2,col=1)
    
    
    fig.update_yaxes(range=[40+vol_min,130-vol_max],row=1,col=1)
    vol_min = vol_min +5
    vol_max = vol_max +15
    
    
    fig.layout.images = [dict(
        source="./GVOL_logo.png",
        xref="paper", yref="paper",
        x=0.98, y=1.1,
        sizex=0.15, sizey=0.15,
        xanchor="center", yanchor="bottom"
      )]

    fig.update_layout(title=symbol+" IV-RV "+str(rv_iv_time.iloc[:,0].name[2:])+" DAY WINDOW",title_x=0.50,
                      yaxis_title="volatility",
                      legend=dict(
                          orientation="h",
                          yanchor="bottom",
                          y=-0.15,
                          xanchor="center",
                          x=0.5
                          ),
                      margin=dict(r=20),
                      template = 'plotly_dark')
    
    fig.show()
    s = s + 1

    
##CHART SUM OF PREMIUM VARIANCE

fig = go.Figure()
s = 0
premium = 0
for time in window:
    rv_iv_time = rv_iv[time]
    premium = rv_iv_time.iloc[:,1].shift(shift[s]) - rv_iv_time.iloc[:,0] + premium
    s = s +1
    
fig.add_trace(go.Bar(x=premium.index,y=premium))
fig.update_traces(marker_color='green', marker_line_color='green',opacity=0.7)




fig.layout.images = [dict(
    source="./GVOL_logo.png",
    xref="paper", yref="paper",
    x=0.98, y=1.1,
    sizex=0.15, sizey=0.15,
    xanchor="center", yanchor="bottom"
  )]


fig.update_layout(title=symbol+" SUM OF VARIANCE PREMIUM",title_x=0.50,
                  yaxis_title="volatility",
                  legend=dict(
                      orientation="h",
                      yanchor="bottom",
                      y=-0.15,
                      xanchor="center",
                      x=0.5
                      ),
                  margin=dict(r=20),
                  template = 'plotly_dark')

fig.show()


In [11]:
now = datetime.datetime.utcnow().date()
now = str(now)

symbols = ('BTC','ETH')
rv_iv = {}

for symbol in symbols:
    

    ##RETRIEVE PRICES
    ###API LITE CLIENTS
    #prices = gvol_client.SpotPrices(symbol=symbol,dateStart='2021-05-15',dateEnd=now)
    
    prices = gvol_client.SpotPricesLite(symbol=symbol)
    prices = pd.json_normalize(prices['SpotPricesLite'])
    prices['date'] = pd.to_datetime(prices['date'],unit='ms')
    prices.set_index('date',inplace=True)
    prices = prices
    prices.columns = ['Curreny','Open','High','Low','Close']
    prices.sort_index(inplace=True)


    ##RETRIEVE CONSTANT ATM IV
    now = datetime.datetime.utcnow().date() + datetime.timedelta(days=1)
    now = str(now)
    ###API LITE CLIENTS
    iv = gvol_client.FixedMaturityAtm(exchange='deribit',symbol=symbol)
    iv = pd.json_normalize(iv['FixedMaturityAtm'])
    iv['date']= pd.to_datetime(iv['date'],unit='ms')
    iv.set_index('date',inplace=True)
    iv = iv.resample('1D').mean()

    ##CALCULATING RV CHOOSE YOUR ESTIMATOR HERE
    window = (7,30,60,90,180)
    ###[raw,parkinson,garmanklass,yangzhang]
    vol_estimator = parkinson

    df = pd.DataFrame()
    for days in window:
        df_days = vol_estimator(prices,window=days,clean=False)*100
        df = pd.concat([df,df_days],axis=1)

    rv = df
    col_names = ["rv" + str(day) for day in window]
    rv.columns = col_names

    #Using dictionary for assign and storing symbol name as dataframe
    rv_iv[symbol] = rv.join(iv)

In [12]:
##NOT VARIABLES DO NOT CHANGE
window = (['rv7','atm7'],['rv30','atm30'],['rv90','atm90'],['rv180','atm180'])

## COMMENT/DECOMMENT IF YOU WANNA SHIFT IV FORWARD
#shift = (7,30,90,180)
shift = (0,0,0,0)
###

s = 0
for time in (window):
    
    #DEFINE SUBPLOTS CHART
    fig = make_subplots(rows=2,cols=1,row_heights=[0.8, 0.2])
    
    rv_iv_time = rv_iv['ETH'][time] - rv_iv['BTC'][time]
    rv_iv_time = rv_iv_time.iloc[-30:]
    
    fig.add_trace(go.Scatter(x=rv_iv_time.index,y=rv_iv_time.iloc[:,0],name=rv_iv_time.iloc[:,0].name,fill='tozeroy'),row=1,col=1)
    fig.add_trace(go.Scatter(x=rv_iv_time.index,y=rv_iv_time.iloc[:,1].shift(shift[s]),name=rv_iv_time.iloc[:,1].name,fill='tozeroy'),row=1,col=1)

    premium = rv_iv_time.iloc[:,1].shift(shift[s]) - rv_iv_time.iloc[:,0]
    fig.add_trace(go.Bar(x=premium.index,y=premium,name='var premium'),row=2,col=1)
    
    ##API LITE CLIENTS
    #fig.update_xaxes(range=['2022-4-17',now])
    #fig.update_xaxes(range=['2020-1-1',now])
    
    vol_min = rv_iv_time.iloc[:,1].min()
    vol_max = rv_iv_time.iloc[:,1].max()+25
    #fig.update_yaxes(range=[vol_min,vol_max],row=1,col=1)
    
    
    fig.layout.images = [dict(
        source="./GVOL_logo.png",
        xref="paper", yref="paper",
        x=0.98, y=1.1,
        sizex=0.15, sizey=0.15,
        xanchor="center", yanchor="bottom"
      )]

    fig.update_layout(title="ETH-BTC VOL SPREADS "+str(rv_iv_time.iloc[:,0].name[2:])+" DAY WINDOW",title_x=0.50,
                      legend=dict(
                          orientation="h",
                          yanchor="bottom",
                          y=-0.15,
                          xanchor="center",
                          x=0.5
                          ),
                      margin=dict(r=20),
                      template = 'plotly_dark')
    
    fig.show()
    s = s + 1

    

In [13]:
import plotly.express as px
rv = ['rv7','rv30','rv90','rv180']
iv = ['atm7','atm30','atm90','atm180']

#Since Nov 2019 data are available both iv and rv all maturities
#For percentile distribution analysis I take only part of dataframe
#where both data are present

data = rv_iv['ETH'][rv] - rv_iv['BTC'][rv]
data = data.iloc[-30:]

fig = px.box(data)

fig.add_trace(
    go.Scatter(
        x=data.columns.values,
        y=data.iloc[-1].values,
        mode="lines+markers",
        line=go.scatter.Line(color="grey",dash='dot'),
        showlegend=False)
)

fig.update_layout(showlegend=True)

fig.layout.images = [dict(
    source="./GVOL_logo.png",
    xref="paper", yref="paper",
    x=0.98, y=0.99,
    sizex=0.15, sizey=0.15,
    xanchor="center", yanchor="bottom"
  )]

fig.update_layout(
    title="ETH-BTC RV SPREADS PERCENTILE DISTRIBUTION (30 DAYS)",title_x=0.50,\
    margin=dict(r=20),
    template='plotly_dark')  

fig.update_yaxes(range=[-60,130])
fig.update_xaxes(title=None)
fig.update_yaxes(title=None)

fig.show()

rv = ['rv7','rv30','rv90','rv180']
iv = ['atm7','atm30','atm90','atm180']

data = rv_iv['ETH'][iv] - rv_iv['BTC'][iv]
data = data.iloc[-30:]

fig = px.box(data)

fig.add_trace(
    go.Scatter(
        x=data.columns.values,
        y=data.iloc[-1].values,
        mode="lines+markers",
        line=go.scatter.Line(color="grey",dash='dot'),
        showlegend=False)
)

fig.update_layout(showlegend=True)

fig.layout.images = [dict(
    source="./GVOL_logo.png",
    xref="paper", yref="paper",
    x=0.98, y=0.99,
    sizex=0.15, sizey=0.15,
    xanchor="center", yanchor="bottom"
  )]

fig.update_layout(
    title="ETH-BTC IV SPREADS PERCENTILE DISTRIBUTION (30 DAYS)",title_x=0.50,\
    margin=dict(r=20),
    template='plotly_dark')  

fig.update_yaxes(range=[-30,50])
fig.update_xaxes(title=None)
fig.update_yaxes(title=None)

fig.show()